<a href="https://colab.research.google.com/github/Ryan-S-S/Oficina-de-CNN/blob/main/Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Instalar bibliotecas necessárias (se não tiver)
# !pip install pandas torch torchvision matplotlib scikit-learn opencv-python

import os
import zipfile
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

#**Dataset**: (definição, organização e pré processamento)

## Baixa o dataset compactado e o extrai.

In [19]:
# --- 1. Baixar e Extrair o Dataset ---
# Se você estiver rodando em Google Colab ou ambiente similar, isso funciona.
# Se estiver em um script local, pode precisar adaptar.
print("Baixando o dataset...")
# Verifica se o arquivo zip já existe para evitar download duplicado
if not os.path.exists('kagglecatsanddogs_5340.zip'):
    !wget https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
    print("Download completo.")
else:
    print("Arquivo zip já existe.")

# Verifica se a pasta PetImages já existe para evitar extração duplicada
if not os.path.exists('PetImages'):
    print("Extraindo o dataset...")
    with zipfile.ZipFile('kagglecatsanddogs_5340.zip', 'r') as zip_ref:
        # Extrair apenas as pastas Cat e Dog, pois __MACOSX pode causar problemas
        for member in zip_ref.namelist():
            if "PetImages/Cat/" in member or "PetImages/Dog/" in member:
                 zip_ref.extract(member, '.')
    print("Extração completa.")
else:
    print("Pasta PetImages já existe.")

Baixando o dataset...
Arquivo zip já existe.
Pasta PetImages já existe.


## Percorre as pastas e organiza as lista de imagem e label.

In [20]:
# --- 2. Organizar os Dados ---
print("Organizando os dados...")
base_dir = 'PetImages'
categories = ['Cat', 'Dog']
image_paths = []
labels = []
corrupted_images = [] # Lista para armazenar caminhos de imagens corrompidas

for category in categories:
    path = os.path.join(base_dir, category)
    # Usamos tqdm para ver o progresso ao listar e verificar imagens
    for img_name in tqdm(os.listdir(path), desc=f"Verificando {category}s"):
        img_path = os.path.join(path, img_name)
        # --- 4. Tratar Imagens Corrompidas ---
        try:
            # Tenta abrir a imagem. Se der erro, é provável que esteja corrompida.
            # O .verify() checa a integridade, mas pode não pegar todos os casos.
            # Abrir e fechar é mais robusto.
            img = Image.open(img_path)
            img.verify() # Verifica a integridade do arquivo
            image_paths.append(img_path)
            labels.append(0 if category == 'Cat' else 1) # 0 para Gato, 1 para Cachorro
        except (IOError, SyntaxError) as e:
            #print(f"Imagem corrompida ou ilegível: {img_path}. Erro: {e}")
            corrupted_images.append(img_path)
            # Pass (ignora a imagem corrompida)

print();
print(f"Total de imagens encontradas: {len(image_paths)}")
print(f"Total de imagens corrompidas ignoradas: {len(corrupted_images)}")

# Criar um DataFrame pandas
df = pd.DataFrame({'path': image_paths, 'label': labels})

Organizando os dados...


Verificando Dogs:  96%|█████████▌| 11970/12501 [00:01<00:00, 9053.91it/s]/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Verificando Dogs: 100%|██████████| 12501/12501 [00:01<00:00, 8889.84it/s]


Total de imagens encontradas: 24998
Total de imagens corrompidas ignoradas: 4


## Separa oque vai ser Treino e Teste.

In [21]:
# --- 3. Dividir os Dados ---
# Usar train_test_split para dividir em treino e validação (ex: 80% treino, 20% validação)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

print(f"\nNúmero de imagens para treino: {len(train_df)}")
print(f"Número de imagens para validação: {len(val_df)}")
print(f"Número de imagens para teste: {len(test_df)}")

# Contar o número de gatos e cachorros em cada dataframe
print("\nContagem de Gatos e Cachorros no conjunto de Treino:")
print(train_df['label'].value_counts().rename(index={0: 'Cat', 1: 'Dog'}))

print("\nContagem de Gatos e Cachorros no conjunto de Validação:")
print(val_df['label'].value_counts().rename(index={0: 'Cat', 1: 'Dog'}))

print("\nContagem de Gatos e Cachorros no conjunto de Teste:")
print(test_df['label'].value_counts().rename(index={0: 'Cat', 1: 'Dog'}))


Número de imagens para treino: 15998
Número de imagens para validação: 5000

Contagem de Gatos e Cachorros no conjunto de Treino:
label
Dog    7999
Cat    7999
Name: count, dtype: int64

Contagem de Gatos e Cachorros no conjunto de Validação:
label
Dog    2000
Cat    2000
Name: count, dtype: int64

Contagem de Gatos e Cachorros no conjunto de Teste:
label
Dog    2500
Cat    2500
Name: count, dtype: int64


## Define a Classe **PetImagesDataset** para facilitar o treinamento.

In [22]:
# --- 5. Criar um Dataset Customizado ---
class PetImagesDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]

        try:
            # Abrir a imagem
            image = Image.open(img_path).convert('RGB') # Garantir que é RGB (3 canais)
            # Aplicar transformações
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
             #print(f"Erro ao carregar/transformar imagem {img_path}: {e}")
             return None, None # Retorna None para sinalizar problema

## Define e aplica as transformações de Pré processamento nas imagens.

In [23]:
# --- 6. Definir Transformações ---
# Transformações para treino (inclui aumento de dados)
# Resize para um tamanho um pouco maior, depois RandomCrop para o tamanho final
# ToTensor converte a imagem PIL para Tensor e escala os pixels para [0, 1]
# Normalize usa médias e desvios padrões comuns (ex: ImageNet)

train_transform = transforms.Compose([
    transforms.Resize((70, 70)),
    transforms.RandomCrop((64, 64)),
    transforms.RandomHorizontalFlip(), # Aumento de dados: vira horizontalmente aleatoriamente
    transforms.ToTensor(),
    # Valores de normalização comuns (médias e desvios padrões por canal RGB)
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Transformações para teste (apenas redimensionar e normalizar)
test_transform = transforms.Compose([
    transforms.Resize((64, 64)), # Redimensiona diretamente para o tamanho final
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Transformações para validação (apenas redimensionar e normalizar)
val_transform = transforms.Compose([
    transforms.Resize((64, 64)), # Redimensiona diretamente para o tamanho final
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


# Criar instâncias dos Datasets
train_dataset = PetImagesDataset(train_df, transform=train_transform)
val_dataset = PetImagesDataset(val_df, transform=val_transform)
test_dataset = PetImagesDataset(test_df, transform=test_transform)

## Define os DataLoaders

In [24]:
# --- 7. Criar DataLoaders ---
# DataLoaders ajudam a carregar os dados em batches e embaralhar (no treino)
BATCH_SIZE = 64

# Função para lidar com amostras None no DataLoader (devido a imagens corrompidas tratadas)
def collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch)) # Remove None tuples
    return torch.utils.data.dataloader.default_collate(batch)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, collate_fn=collate_fn)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# **Construir o Modelo CNN**

In [25]:
# --- 8. Definir o Modelo (CNN Simples) ---
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=1): # 1 classe de saída para classificação binária com BCEWithLogitsLoss
        super(SimpleCNN, self).__init__()
        # Camada Convolucional 1
        # Entrada: 3 canais (RGB), Saída: 32 canais, Kernel: 3x3
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Diminui a dimensão espacial pela metade

        # Camada Convolucional 2
        # Entrada: 32 canais, Saída: 64 canais, Kernel: 3x3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Diminui a dimensão espacial pela metade

        # Camada Convolucional 3
        # Entrada: 64 canais, Saída: 128 canais, Kernel: 3x3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # Diminui a dimensão espacial pela metade

        # Camadas Totalmente Conectadas
        # Após as camadas de pooling, a dimensão da imagem 64x64 -> 32x32 -> 16x16 -> 8x8
        # Temos 128 canais, então 128 * 8 * 8 = 8192 features antes da primeira camada FC
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5) # Dropout para regularização
        self.fc2 = nn.Linear(512, num_classes) # Saída: 1 neurônio para classificação binária

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))

        # Achatar (flatten) os dados antes das camadas totalmente conectadas
        x = x.view(-1, 128 * 8 * 8) # -1 infere o tamanho do batch size

        x = self.dropout(self.relu4(self.fc1(x)))
        x = self.fc2(x)
        return x

# Instanciar o modelo
model = SimpleCNN(num_classes=1)

# --- Configurar Dispositivo (CPU ou GPU) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# --- 8. Configurar Função de Perda e Otimizador ---
# BCEWithLogitsLoss é boa para classificação binária, combina Sigmoid + Binary Cross Entropy
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Otimizador Adam

# **Treinamento**

## Tenta carregar o modelo já treinado.

In [26]:
# --- 9. Treinar o Modelo ---
# Se você carregar um modelo salvo na época 5 e NUM_EPOCHS for 10, ele rodará até a época 5 (totalizando 10).
TOTAL_NUM_EPOCHS = 10
NUM_EPOCHS = TOTAL_NUM_EPOCHS

# --- Configurar Salvamento do Melhor Modelo ---
best_model_path = 'best_cats_vs_dogs_model.pth' # Nome do arquivo onde o modelo será salvo/carregado

# Inicializar a época de início do treinamento
start_epoch = 0
# Inicializar a melhor acurácia (será substituída se um modelo for carregado)
best_accuracy = -1.0 # Inicializa aqui, antes da tentativa de carregar

# --- Lógica para Carregar o Modelo Salvo (se existir) ---
if os.path.exists(best_model_path):
    print(f"Arquivo de modelo encontrado em {best_model_path}. Carregando...")
    try:
        # Carregar o checkpoint (inclui estado do modelo, otimizador, época e melhor acurácia)
        checkpoint = torch.load(best_model_path, map_location=device)

        # Carregar o estado do modelo
        model.load_state_dict(checkpoint['model_state_dict'])

        # Carregar o estado do otimizador (importante para continuar o treinamento)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        # Carregar a melhor acurácia salva
        best_accuracy = checkpoint['best_accuracy']

        # Definir a época de início para continuar de onde parou
        start_epoch = checkpoint['epoch'] + 1

        print(f"Modelo e otimizador carregados com sucesso. Retomando o treinamento a partir da Época {start_epoch}.")
        print(f"Melhor acurácia de validação prévia: {best_accuracy:.4f}")

    except Exception as e:
        print(f"Erro ao carregar o modelo de {best_model_path}. Começando do zero. Erro: {e}")
        # Se houver erro ao carregar, start_epoch e best_accuracy permanecem nos valores iniciais (0 e -1.0)
else:
    print(f"Nenhum arquivo de modelo encontrado em {best_model_path}. Iniciando treinamento do zero.")


Arquivo de modelo encontrado em best_cats_vs_dogs_model.pth. Carregando...
Modelo e otimizador carregados com sucesso. Retomando o treinamento a partir da Época 10.
Melhor acurácia de validação prévia: 0.8767


## O fluxo do treinamento.

In [27]:
# --- Treinamento---
print("\nIniciando o treinamento...")

for epoch in range(start_epoch, NUM_EPOCHS):
    model.train() # Coloca o modelo em modo de treino (habilita dropout, etc.)
    running_loss = 0.0

    # Loop sobre os batches do treino
    for images, labels in tqdm(train_loader, desc=f"Época {epoch+1}/{NUM_EPOCHS} [Treino]"): # Ajusta o desc para mostrar o número correto da época
        # Mover dados para o dispositivo correto (CPU/GPU)
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1) # labels precisam ser float e ter a mesma dimensão da saída do modelo

        # Zerar os gradientes do otimizador
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calcular a perda
        loss = criterion(outputs, labels)

        # Backward pass e otimização
        loss.backward() # Calcula os gradientes
        optimizer.step() # Atualiza os pesos

        running_loss += loss.item() * images.size(0)

    # --- Avaliação no conjunto de validação após cada época ---
    model.eval() # Coloca o modelo em modo de avaliação (desabilita dropout, etc.)
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad(): # Desabilita o cálculo de gradientes na avaliação
        for images, labels in tqdm(val_loader, desc=f"Época {epoch+1}/{NUM_EPOCHS} [Validação]"): # ADIÇÃO: Ajusta o desc para mostrar o número correto da época
            # Mover dados para o dispositivo correto
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            # Forward pass
            outputs = model(images)

            # Calcular a perda de validação
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            # Calcular acurácia
            predicted = torch.round(torch.sigmoid(outputs))
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_val_loss = val_loss / len(val_loader.dataset)
    epoch_accuracy = correct_predictions / total_samples


    # --- Lógica para Salvar o Melhor Modelo (Checkpoint Completo) ---
    # Se a acurácia nesta época for maior que a melhor acurácia vista até agora
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy # Atualiza a melhor acurácia
        # Salva um dicionário (checkpoint) com mais informações
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_accuracy': best_accuracy
        }
        torch.save(checkpoint, best_model_path)
        print(f"-> Salvo novo melhor modelo (checkpoint) em {best_model_path} com Acurácia de Validação: {best_accuracy:.4f}")


    print(f"Época [{epoch+1}/{NUM_EPOCHS}], Perda de Treino: {epoch_loss:.4f}, Perda de Validação: {epoch_val_loss:.4f}, Acurácia de Validação: {epoch_accuracy:.4f}")

print("\nTreinamento finalizado!")


Iniciando o treinamento...

Treinamento finalizado!


# **Avaliação e Métricas**

In [28]:
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [29]:

# --- 10. Avaliação Final ---
print("\nAvaliação final no conjunto de validação...")
model.eval()
val_loss = 0.0
correct_predictions = 0
total_samples = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Avaliando"):
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)

        predicted = torch.round(torch.sigmoid(outputs))
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())


final_val_loss = val_loss / len(test_loader.dataset)
final_accuracy = correct_predictions / total_samples

print(f"Perda Final de Validação: {final_val_loss:.4f}")
print(f"Acurácia Final de Validação: {final_accuracy:.4f}")

# Você pode gerar um relatório de classificação mais detalhado se quiser
from sklearn.metrics import classification_report
print("\nRelatório de Classificação:")
print(classification_report(all_labels, all_predictions, target_names=['Cat', 'Dog']))


Avaliação final no conjunto de validação...


Avaliando:  73%|███████▎  | 58/79 [00:26<00:07,  2.83it/s]/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Avaliando: 100%|██████████| 79/79 [00:33<00:00,  2.39it/s]

Perda Final de Validação: 0.2832
Acurácia Final de Validação: 0.8772

Relatório de Classificação:
              precision    recall  f1-score   support

         Cat       0.86      0.90      0.88      2500
         Dog       0.90      0.85      0.87      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



# **Uso de Imagem própria**

In [30]:
# --- Bloco para Testar com uma Foto Única ---

# 1. Defina o caminho para a sua foto
caminho_da_sua_foto = '/content/cachorro.jpg'

# Verifique se o arquivo existe
if not os.path.exists(caminho_da_sua_foto):
    print(f"Erro: Arquivo não encontrado em {caminho_da_sua_foto}")
else:
    try:
        # 2. Carregar a imagem
        imagem = Image.open(caminho_da_sua_foto).convert('RGB') # Garante 3 canais

        # Exibir a imagem original
        print("Imagem Original:")
        display(imagem)


        # 3. Aplicar as transformações de validação
        imagem_tensor = val_transform(imagem)

        # ADIÇÃO: Exibir a imagem após as transformações (como tensor)
        # Para exibir, precisamos converter de volta para PIL Image.
        # Primeiro, desnormalizamos.
        # Os valores de normalização foram (0.485, 0.456, 0.406) e (0.229, 0.224, 0.225)
        # Imagem_normalizada = (Imagem_original - Média) / StdDev
        # Imagem_original = Imagem_normalizada * StdDev + Média
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        imagem_denormalizada = imagem_tensor * std + mean

        # Clipa os valores para garantir que estejam no intervalo [0, 1]
        imagem_denormalizada = torch.clamp(imagem_denormalizada, 0, 1)

        # Converte de tensor para PIL Image (espera CHW, Converte para HWC e depois para PIL)
        imagem_transformada_pil = transforms.ToPILImage()(imagem_denormalizada)

        print("\nImagem Após Transformações de Validação:")
        display(imagem_transformada_pil)


        # 4. Preparar o tensor para o modelo
        imagem_tensor = imagem_tensor.unsqueeze(0)
        imagem_tensor = imagem_tensor.to(device)

        # 5. Colocar o modelo em modo de avaliação
        model.eval()

        # 6. Passar a imagem pelo modelo e 7. Interpretar a saída
        with torch.no_grad(): # Desabilita o cálculo de gradientes para inferência (mais rápido e economiza memória)
            output = model(imagem_tensor)

            # A saída do modelo são logits.
            # aplicamos Sigmoid para obter a probabilidade e depois arredondamos para a classe
            probability = torch.sigmoid(output).item() # .item() pega o valor escalar do tensor

            # Se a probabilidade for >= 0.5, prevemos 1 (Cachorro), caso contrário 0 (Gato)
            predicted_class_index = round(probability)

            # Mapear o índice de volta para o nome da classe
            predicted_class_name = "Dog" if predicted_class_index == 1 else "Cat"

        # Exibir o resultado
        print(f"\nAnálise da imagem: {caminho_da_sua_foto}")
        print(f"Probabilidade (Dog): {probability:.4f}")
        print(f"Probabilidade (Cat): {1 - probability:.4f}") # Probabilidade de Cat é 1 - Probabilidade de Dog
        print(f"O modelo prevê que a imagem é um: {predicted_class_name}")

    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {caminho_da_sua_foto}")
    except Exception as e:
        print(f"Ocorreu um erro ao processar a imagem: {e}")

Erro: Arquivo não encontrado em /content/cachorro.jpg
